## 步驟 1: 環境設定與授權

請在此處選擇您的操作模式、程式碼分支和啟動模式，然後點擊「確認設定並掛載雲端硬碟」。

- **操作模式**: 
  - `NORMAL`：適用於一般使用者，提供穩定的操作體驗。
  - `DEBUG`：適用於開發者，啟用額外的日誌記錄和測試功能。
- **程式碼分支**:
  - `main`：主要穩定分支。
  - `dev`：開發分支，包含最新但可能不穩定的功能。
- **啟動模式**:
  - `AUTO`：自動完成所有設定和啟動步驟。
  - `MANUAL`：需要手動執行後續儲存格。

In [ ]:
# --- Python 模組導入 ---
import os
import subprocess
import requests
from google.colab import drive
import logging

# --- 環境變數與設定 ---
REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git"
REPO_PATH = "/content/wolfAI_v1"
LOG_FILE = "/content/colab_runner.log"

#@title 確認設定並掛載雲端硬碟
#@markdown ### **1. 參數設定**
OPERATION_MODE = "NORMAL"  #@param ["NORMAL", "DEBUG"]
BRANCH = "main"  #@param ["main", "dev"]
START_MODE = "AUTO"  #@param ["AUTO", "MANUAL"]

# --- 日誌設定 ---
logging.basicConfig(
    level=logging.DEBUG if OPERATION_MODE == 'DEBUG' else logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ]
)

def run_command(command, cwd=None):
    """執行 shell 命令並記錄輸出"""
    logging.info(f"Executing command: {' '.join(command)}")
    try:
        process = subprocess.Popen(
            command, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT, 
            text=True,
            cwd=cwd
        )
        for line in iter(process.stdout.readline, ''):
            logging.info(line.strip())
        process.stdout.close()
        return_code = process.wait()
        if return_code:
            raise subprocess.CalledProcessError(return_code, command)
    except Exception as e:
        logging.error(f"Command failed: {e}")
        raise

def setup_environment():
    """設定 Colab 環境，包括掛載雲端、克隆 repo 等"""
    logging.info("--- 步驟 1: 環境設定開始 ---")
    
    # 掛載 Google Drive
    logging.info("Mounting Google Drive...")
    drive.mount('/content/drive')
    logging.info("Google Drive mounted successfully.")
    
    # 克隆或更新程式碼倉庫
    if not os.path.exists(REPO_PATH):
        logging.info(f"Cloning repository from {REPO_URL}...")
        run_command(['git', 'clone', '--branch', BRANCH, REPO_URL, REPO_PATH])
    else:
        logging.info("Repository exists. Fetching updates...")
        run_command(['git', 'fetch', '--all'], cwd=REPO_PATH)
        run_command(['git', 'checkout', BRANCH], cwd=REPO_PATH)
        run_command(['git', 'pull', 'origin', BRANCH], cwd=REPO_PATH)
    
    logging.info("--- 環境設定完成 ---")

# --- 主執行流程 ---
try:
    setup_environment()
except Exception as e:
    logging.critical(f"An unrecoverable error occurred during setup: {e}")
    # 在此處可以加入錯誤通知機制


## 步驟 2: 安裝依賴套件

此儲存格將根據 `backend/requirements.txt` 安裝所有必要的 Python 套件。

在 `AUTO` 模式下，此儲存格將自動執行。

In [ ]:
def install_dependencies():
    """安裝專案依賴"""
    logging.info("--- 步驟 2: 安裝依賴套件開始 ---")
    requirements_path = os.path.join(REPO_PATH, 'backend', 'requirements.txt')
    if os.path.exists(requirements_path):
        run_command(['pip', 'install', '-r', requirements_path])
        logging.info("--- 依賴套件安裝完成 ---")
    else:
        logging.error(f"requirements.txt not found at {requirements_path}")
        raise FileNotFoundError(f"requirements.txt not found at {requirements_path}")

if START_MODE == 'AUTO':
    try:
        install_dependencies()
    except Exception as e:
        logging.critical(f"An unrecoverable error occurred during dependency installation: {e}")

## 步驟 3: 執行系統健康檢查 (僅在 DEBUG 模式)

如果操作模式設定為 `DEBUG`，此儲存格將執行自動化測試 (`pytest`) 來驗證後端服務的正確性。

在 `AUTO` 模式下，此儲存格將自動執行。

In [ ]:
def run_health_checks():
    """執行 pytest 測試"""
    logging.info("--- 步驟 3: 執行系統健康檢查開始 ---")
    backend_path = os.path.join(REPO_PATH, 'backend')
    run_command(['pytest'], cwd=backend_path)
    logging.info("--- 系統健康檢查完成 ---")
    
if START_MODE == 'AUTO' and OPERATION_MODE == 'DEBUG':
    try:
        run_health_checks()
    except Exception as e:
        logging.error(f"Health checks failed: {e}")

## 步驟 4: 啟動後端與前端服務

此儲存格將使用 `start.sh` 腳本啟動 FastAPI 後端伺服器和 Next.js 前端應用程式。

在 `AUTO` 模式下，此儲存格將自動執行。

In [ ]:
def start_services():
    """啟動後端與前端服務"""
    logging.info("--- 步驟 4: 啟動服務開始 ---")
    start_script_path = os.path.join(REPO_PATH, 'scripts', 'start.sh')
    if os.path.exists(start_script_path):
        # 賦予腳本執行權限
        run_command(['chmod', '+x', start_script_path])
        
        # 執行腳本
        # 注意：在 Colab 中直接啟動長時間運行的服務可能會很複雜
        # 這裡我們使用背景執行的方式
        logging.info("Starting services in the background...")
        env = os.environ.copy()
        env['OPERATION_MODE'] = OPERATION_MODE
        
        process = subprocess.Popen(
            [start_script_path],
            env=env,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            cwd=os.path.join(REPO_PATH)
        )
        
        logging.info("Services are starting. Check backend.log and frontend.log for details.")
        # 為了演示，我們不會等待此進程結束
        # 在實際應用中，您可能需要更複雜的進程管理
    else:
        logging.error(f"start.sh not found at {start_script_path}")
        raise FileNotFoundError(f"start.sh not found at {start_script_path}")

if START_MODE == 'AUTO':
    try:
        start_services()
        # 顯示日誌文件的最後幾行
        print("\n--- Displaying last 10 lines of backend.log ---")
        !tail -n 10 /content/wolfAI_v1/backend.log
        print("\n--- Displaying last 10 lines of frontend.log ---")
        !tail -n 10 /content/wolfAI_v1/frontend.log
    except Exception as e:
        logging.critical(f"Failed to start services: {e}")